In [1]:
import networkx as nx

In [4]:
G_big = nx.read_gpickle('../data/playlists_projection.gpkl')
selected_edges = [(u,v) for u,v in G.edges() if G.get_edge_data(u,v)['weight']>4]
subg = G.edge_subgraph(selected_edges)

In [5]:
big_connected = subg.subgraph(list(nx.connected_components(subg))[0]).copy()

In [4]:
len(big_connected)

5653

In [12]:
nodes = big_connected.nodes()

In [6]:
import pandas as pd

In [7]:
# nx.get_node_attributes(G, '37i9dQZF1DX0suvPmTqswW')

In [8]:
playlists = pd.read_pickle('../data/final_spotify_playlists.pkl')

In [23]:
playlist_data = pd.DataFrame(playlists.loc[list(nodes)], columns=['name'])

In [24]:
categories = pd.read_csv('../data/playlists_categories.csv').set_index('id')

In [25]:
categories_filtered = categories[categories.index.isin(playlist_data.index)]

In [26]:
playlist_data.loc[categories_filtered.index, 'category']=categories_filtered.values

In [27]:
playlist_data['category_source']=False

In [28]:
playlist_data.loc[categories_filtered.index, 'category_source']=True

In [70]:
del  playlist_data['category_sort']

In [71]:
playlist_data.head()

,name,category,category_source
37i9dQZF1DXaKIA8E7WcJj,All Out 60s,soul,False
37i9dQZF1DXdHPp93Fnfds,Mujeres del Indie,latin,False
37i9dQZF1DWVRje2pMSCzM,Brasil Anos 80,rock,False
37i9dQZF1DXbBKF9yWSvWR,A Shot Of Indie,chill,False
37i9dQZF1DX8bPpooSiXy2,Tropical Christmas,at_home,False


In [30]:
isnull = playlist_data['category'].isnull()
while np.sum(isnull)>0:
    dict_infected = {}
    # we practice a bit of socialism with the category counts
    # priorizing those with less nodes in case of case of equal value
    category_sort = playlist_data['category'].value_counts().sort_values(ascending=True)
    playlist_data['category_sort']=playlist_data['category'].apply(lambda x: category_sort[x] if not pd.isnull(x) else pd.np.nan)
    for i, row in playlist_data[-isnull].sort_values(by='category_sort').iterrows():
        for n in big_connected.neighbors(i):
            if isnull[n]:
                weight = G.get_edge_data(i,n)['weight']
                if n not in dict_infected:
                    dict_infected[n]={row['category']:weight}
                else:
                    if row['category'] not in dict_infected[n]:
                        dict_infected[n][row['category']]=0
                    dict_infected[n][row['category']]+=weight
    for i, dict_inf_node in dict_infected.items():
        # we normalize by the total amount of nodes of that category
        normalized_sort = {k:v/category_sort[k] for k,v in dict_inf_node.items()}
        category_i = max(normalized_sort, key=normalized_sort.get)
        playlist_data.loc[i, 'category']=category_i
        category_sort[category_i]+=1
    isnull = playlist_data['category'].isnull()
        

C:\Users\berna\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """


5216 3547
1669 1326
343 277
66 60
6 5
1 1


In [34]:
# dict_infected
# playlist_data['category'].value_counts()

In [32]:
playlist_data[['name','category','category_source']]

,name,category,category_source
37i9dQZF1DXaKIA8E7WcJj,All Out 60s,soul,False
37i9dQZF1DXdHPp93Fnfds,Mujeres del Indie,latin,False
37i9dQZF1DWVRje2pMSCzM,Brasil Anos 80,rock,False
37i9dQZF1DXbBKF9yWSvWR,A Shot Of Indie,chill,False
37i9dQZF1DX8bPpooSiXy2,Tropical Christmas,at_home,False
...,...,...,...
37i9dQZF1DWVnKyBKrE7lN,Most Streamed Songs of All Time,gaming,False
37i9dQZF1DWWu1NXpHyPJK,90's Allerbeste,romance,False
37i9dQZF1DX0suvPmTqswW,Barnsånger i farten,at_home,False
37i9dQZF1DX9C8KzGEUKV4,Éxitos Perú,latin,False


In [35]:
for i in ['name','category','category_source']:
    nx.set_node_attributes(big_connected, playlist_data[i].to_dict(), i)
# max(stats, key=stats.get)

In [43]:
for (n,d) in big_connected.nodes(data=True):
        del d["bipartite"]

In [28]:
nx.set_node_attributes(big_connected, playlist_data['name'].to_dict(), 'name')

In [39]:
# nx.get_node_attributes(big_connected, '37i9dQZF1DXaKIA8E7WcJj')

In [44]:
big_connected.nodes['37i9dQZF1DXaKIA8E7WcJj']

{'name': 'All Out 60s', 'category': 'soul', 'category_source': False}

In [47]:
# nx.node_link_data(big_connected)

In [67]:
nodes = list(big_connected.nodes(data=True))

In [64]:
nodes_map = {i: for i,v in enumerate(nodes)}

In [65]:
edges_light = [(nodes_map[u], nodes_map[v]) for u,v in big_connected.edges()]

In [68]:
# json_obj = nx.node_link_data(big_connected)
with open('big_graph_nodes.json', 'w') as f:
    json.dump(nodes, f)

In [59]:
# big_connected.edges()
adj = nx.adjacency_matrix(big_connected)

In [61]:
adj.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

<5653x5653 sparse matrix of type '<class 'numpy.intc'>'
	with 744920 stored elements in Compressed Sparse Row format>

C:\Users\berna\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """Entry point for launching an IPython kernel.


In [161]:
playlist_data['category_sort'][-isnull]

37i9dQZF1DX6G79ZQrQjvV    13.0
37i9dQZF1DWUNIrSzKgQbP    46.0
37i9dQZF1DXcr2UzLGERUU    25.0
37i9dQZF1DX1gjl24GAQC0     6.0
37i9dQZF1DX7ZqKP81pHlu    19.0
                          ... 
37i9dQZF1DWVEcPtggpQMu    29.0
37i9dQZF1DWUqr1os0aCxH    42.0
37i9dQZF1DX5oYC8mRiXUR     5.0
37i9dQZF1DX5J7FIl4q56G    12.0
37i9dQZF1DX6BChUV4Ga35    25.0
Name: category_sort, Length: 437, dtype: float64

In [146]:
playlist_data['category'].sort_values()

37i9dQZF1DXaKIA8E7WcJj    NaN
37i9dQZF1DXdHPp93Fnfds    NaN
37i9dQZF1DWVRje2pMSCzM    NaN
37i9dQZF1DXbBKF9yWSvWR    NaN
37i9dQZF1DX8bPpooSiXy2    NaN
                         ... 
37i9dQZF1DWVnKyBKrE7lN    NaN
37i9dQZF1DWWu1NXpHyPJK    NaN
37i9dQZF1DX0suvPmTqswW    NaN
37i9dQZF1DX9C8KzGEUKV4    NaN
37i9dQZF1DX7QVY9I8NGmC    NaN
Name: category, Length: 5653, dtype: object

array([ 1,  1,  2,  3,  3,  4,  4,  4,  5,  5,  5,  6,  6,  6,  7,  8,  8,
        9,  9, 10, 10, 10, 10, 12, 12, 13, 13, 19, 25, 26, 27, 29, 37, 42,
       46], dtype=int64)

In [140]:
dict_infected['37i9dQZF1DX47UHGK8zGt8']

{'pride': 13,
 'family': 15,
 'roots': 17,
 'travel': 6,
 'toplists': 18,
 'mood': 9,
 'at_home': 30,
 'romance': 27,
 'dinner': 10,
 'pop': 13,
 'party': 6,
 'gaming': 5,
 'chill': 9,
 'workout': 5}

In [138]:
{k:v for dict_infected}

SyntaxError: invalid syntax (<ipython-input-138-f7cccbb62874>, line 1)

In [129]:
isnull = playlist_data['category'].isnull()

In [59]:
{}

False

In [65]:
[i for i in G.neighbors('37i9dQZF1DX6G79ZQrQjvV') if not isnull[i]]

['37i9dQZF1DXcr2UzLGERUU',
 '37i9dQZF1DX5nluNjMjoM0',
 '37i9dQZF1DXadOVCgGhS7j',
 '37i9dQZF1DXauMQRH2JBCJ',
 '37i9dQZF1DWTMR78LDoAZC',
 '37i9dQZEVXbLnolsZ8PSNw',
 '37i9dQZF1DXbHwYW9IUCXA',
 '37i9dQZF1DWU8quswnFt3c',
 '37i9dQZF1DX0dClrg1J8Br',
 '37i9dQZF1DWYtpM7d4h5Xa',
 '37i9dQZF1DX5gQonLbZD9s',
 '37i9dQZF1DX7rOY2tZUw1k',
 '37i9dQZF1DWUa8ZRTfalHk',
 '37i9dQZF1DX0q1RHoDiZBg',
 '37i9dQZF1DX4kJGZhtnOIh',
 '37i9dQZF1DWYs2pvwxWA7l',
 '37i9dQZF1DWTLrNDPW5co2',
 '37i9dQZF1DWZNJXX2UeBij',
 '37i9dQZF1DWUxdwkOJZYCJ',
 '37i9dQZF1DWUoqEG4WY6ce',
 '37i9dQZF1DWVmps5U8gHNv',
 '37i9dQZF1DWXnscMH24yOc',
 '37i9dQZF1DX8a1tdzq5tbM',
 '37i9dQZF1DWV3VLITCZusq',
 '37i9dQZF1DWXmlLSKkfdAk',
 '37i9dQZF1DXbmCTvLQy5AO',
 '37i9dQZF1DXdsiL9gD4vAA',
 '37i9dQZF1DWTLSN7iG21yC',
 '37i9dQZF1DX4pAtJteyweQ',
 '37i9dQZF1DX8C9xQcOrE6T',
 '37i9dQZF1DX0IyMQV27EGn',
 '37i9dQZF1DWZIQpJDqCc10',
 '37i9dQZF1DXd4bJEFQJTXh',
 '37i9dQZF1DX889U0CL85jj',
 '37i9dQZF1DX76Wlfdnj7AP',
 '37i9dQZF1DX873GaRGUmPl',
 '37i9dQZF1DX4W3aJJYCDfV',
 

In [55]:
nx.is_connected(G)

True

In [67]:
G.get_edge_data('37i9dQZF1DXcr2UzLGERUU', '37i9dQZF1DX6G79ZQrQjvV')['weight']

1

In [98]:
es)

In [99]:
len(selected_edges)

372939

In [103]:
subg

In [101]:
[len(k) for k in]

[5653,
 6,
 12,
 4,
 2,
 4,
 2,
 2,
 36,
 2,
 2,
 5,
 2,
 2,
 3,
 15,
 3,
 2,
 2,
 2,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2]

In [102]:
connected = list(nx.connected_components(subg))

In [97]:
connected[4]

{'37i9dQZF1DWT0d3wigiTss',
 '37i9dQZF1DWTpJIpOxN3r8',
 '37i9dQZF1DWUKPeBypcpcP',
 '37i9dQZF1DWV90ZWj21ygB',
 '37i9dQZF1DWVEt8B7a1H1M',
 '37i9dQZF1DWVy7RMetNOoa',
 '37i9dQZF1DWWSads6V2oIk',
 '37i9dQZF1DWXzR2GKEiHgT',
 '37i9dQZF1DWYFCj3KyU0r0',
 '37i9dQZF1DWYgToHlKhwA5',
 '37i9dQZF1DWZoPCylYnvtU',
 '37i9dQZF1DWZyr63wC7yP0',
 '37i9dQZF1DX0ES2mnOVvai',
 '37i9dQZF1DX0IWthTyxilL',
 '37i9dQZF1DX1jk5JfpZoMf',
 '37i9dQZF1DX2mFmJUZg4Mp',
 '37i9dQZF1DX3f9U3gq0DBT',
 '37i9dQZF1DX4PP3DA4J0N8',
 '37i9dQZF1DX4aYNO8X5RpR',
 '37i9dQZF1DX5ZTdvWvQHAL',
 '37i9dQZF1DX6EE01mw9tEh',
 '37i9dQZF1DX6mXxqnWLIPb',
 '37i9dQZF1DX709Kv0V067V',
 '37i9dQZF1DX7Mv4BQ3Z4nC',
 '37i9dQZF1DX8KZtFJYy6Ev',
 '37i9dQZF1DX8OfwZVxJpWy',
 '37i9dQZF1DX8ymr6UES7vc',
 '37i9dQZF1DX9NmDLwNQnXE',
 '37i9dQZF1DX9if5QDLdzCa',
 '37i9dQZF1DXaHXkx4nO9cm',
 '37i9dQZF1DXabJG3i5q2yk',
 '37i9dQZF1DXavtmWzC1MpQ',
 '37i9dQZF1DXaw68inx4UiN',
 '37i9dQZF1DXbcPC6Vvqudd',
 '37i9dQZF1DXbexOT9jE7S1',
 '37i9dQZF1DXdp5bwJ1FHFe',
 '6YMK6nqc1jS11fGraCU1o2'}